# collaborative item based Recommender


In [1]:
import pandas as pd
from scipy import sparse # shrink filesize of df
# pw will give us distances (0...n) based on cosine similarity
from sklearn.metrics.pairwise import pairwise_distances

In [2]:
from books_helper import Helper;
helper = Helper()

in constructor1


In [3]:
books=  pd.read_csv("../data/books.csv")
print( f"shape of the books dataframe is {books.shape}")
books.head(1)

shape of the books dataframe is (10000, 26)


,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,description,num_pages,is_ebook
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,"Could you survive on your own, in the wild, wi...",374.0,False


In [7]:
ratings = pd.read_csv("../data/ratings.csv")
print( f"shape of the books dataframe is {ratings.shape}")
ratings.head(1)

shape of the books dataframe is (5976479, 3)


,user_id,book_id,rating
0,1,258,5


In [8]:
len(ratings['user_id'].unique()),len(to_read['user_id'].unique())
len(set(ratings['user_id']) - set(to_read['user_id'])),
len(set(to_read['user_id']) - set(ratings['user_id']))

NameError: name 'to_read' is not defined

In [ ]:
books =pd.merge(books[['book_id','title']],ratings, how ='inner',left_on ='book_id',right_on='book_id')
books.head(1)

In [9]:
ratings.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [7]:
len(ratings['book_id'].unique()),len(ratings['user_id'].unique())
# we  have. double the number of books than the users so , there is a very good chance that
# each user has read multiple books and they have over lap.

(10000, 53424)

In [10]:
# Find out if users have done  multiple book reviews?
ratings.groupby('user_id')['book_id'].value_counts()

user_id  book_id
1        4          1
         10         1
         11         1
         13         1
         16         1
                   ..
53424    8609       1
         8726       1
         8969       1
         9255       1
         9849       1
Name: book_id, Length: 5976479, dtype: int64

In [11]:
# Find out if users have done  multiple book reviews?
ratings.groupby('book_id')['user_id'].count().sort_values(ascending=False)

book_id
1       22806
2       21850
4       19088
3       16931
5       16604
        ...  
9315       36
1935       33
9486       24
9345       11
7803        8
Name: user_id, Length: 10000, dtype: int64

In [13]:
# Are there any books that we don't have ratings for?
print(f" Difference in books in ratings and books dataset {len(set(books['book_id']) - set(ratings['book_id']))}.")
print(f" Out of {len(set(books['book_id']))} books we have ratings for all {len(set(ratings['book_id']))} books.")
      

 Difference in books in ratings and books dataset 0.
 Out of 10000 books we have ratings for all 10000 books.


,book_id,title,user_id,rating
0,1,"The Hunger Games (The Hunger Games, #1)",2886,5


###### Create pivot table
---

Because we're creating an item-based collaborative recommender (where item in this case is our books), we'll set up our pivot table as follows:
1. The `title` will be the index
2. The `userId` will be the column
3. The `rating` will be the value

In [16]:
pivot =df.pivot_table(index ='title',columns ='user_id',values='rating')
pivot

user_id,1,2,3,4,5,6,7,8,9,10,...,53415,53416,53417,53418,53419,53420,53421,53422,53423,53424
title,,,,,,,,,,,,,,,,,,,,,
"Angels (Walsh Family, #3)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""حكايات فرغلي المستكاوي ""حكايتى مع كفر السحلاوية",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#GIRLBOSS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
"'Tis (Frank McCourt, #2)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
واحة الغروب,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
يوتوبيا,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ڤيرتيجو,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
#turn the matrix in to  sparse matrix
pivot_sparse = sparse.csr_matrix(pivot.fillna(0))
pivot_sparse.shape

(9964, 53424)

In [18]:
import sys
sys.getsizeof(pivot),sys.getsizeof(pivot_sparse)

(4259434153, 64)

In [ ]:
# Feed sparse matrix to pair wise disance to calculate. distance between every book
recommender =pairwise_distances(pivot_sparse,metric='cosine')

In [ ]:
#Create distance dataframe
recommender_df = pd.DataFrame(recommender , index =pivot.index,columns =pivot.index)
recommender_df.head()

In [ ]:
recommender_df['The Great Gatsby'].sort_values()[1:5]

In [9]:
helper.find_titles(books,title ="positive")

['The Power of Positive Thinking',
 'The Happiness Advantage: The Seven Principles of Positive Psychology That Fuel Success and Performance at Work']

In [ ]:
my_fav_books = ['The Alchemist','The Adventures of Sherlock Holmes','The Great Gatsby','To Kill a Mockingbird','The Da Vinci Code (Robert Langdon, #2)','The Fellowship of the Ring (The Lord of the Rings, #1)']

###### References
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html

https://www.kaggle.com/emilytries/plotly-library-in-goodreads-books-dataset

https://towardsdatascience.com/my-journey-to-building-book-recommendation-system-5ec959c41847